## Targeted Marketing Agent

This notebook guides you through the steps of creating an AI Agent that will create targeted social media campaigns for hotels.

### Step 1: Create tools for AI Agent

Follow the instructions in each cell below to generate the tools - user-defined functions in this case - to be used by the agent to help it generate the targeted social media posts.

Each cell will contain instructions and sql or python code to execute. Ensure to execute the cells in order sequentially.


In [0]:
# Create Databricks widgets to store names and values to be used throughout this Notebook

# These are placeholder, please use the UI form fields at the top of this Notebook to enter in their values
dbutils.widgets.text("warehouse_id", "")
dbutils.widgets.text("catalog", "")
dbutils.widgets.text("database", "")
dbutils.widgets.text("hotel_to_promote", "")

dbutils.widgets.dropdown("hotel_class",
    "Resort", # Default value
    [
    "Resort",
    "Extended Stay",
    "Luxury",
    "Economy",
    "Airport"
])


In [0]:
%sql
-- This function takes a hotel class as an input and finds the lowest performing hotel in that class that has at least 3 customer reviews and has an above-average customer satisfaction rating

USE CATALOG IDENTIFIER(:catalog);
USE DATABASE IDENTIFIER(:database);

CREATE OR REPLACE FUNCTION get_hotel_to_promote (input_hotel_class STRING COMMENT 'Hotel class to filter by')
RETURNS TABLE (
  hotel_id STRING,
  hotel_name STRING,
  hotel_city STRING,
  hotel_country STRING,
  hotel_description STRING,
  hotel_class STRING,
  average_review_rating DOUBLE,
  review_count INT
)
LANGUAGE SQL
COMMENT 'This function takes a hotel class as an input and finds the lowest performing hotel in that class that has at least 3 customer reviews and has an above-average customer satisfaction rating'
RETURN
  (SELECT
    `HOTEL_ID`,
    `HOTEL_NAME`,
    `HOTEL_CITY`,
    `HOTEL_COUNTRY`,
    `HOTEL_DESCRIPTION`,
    `HOTEL_CLASS`,
    `AVERAGE_REVIEW_RATING`,
    `REVIEW_COUNT`
    FROM
      hotel_stats
    WHERE HOTEL_CLASS = input_hotel_class
    AND `REVIEW_COUNT` > 2
    AND `AVERAGE_REVIEW_RATING` > (
      SELECT
        AVG(`AVERAGE_REVIEW_RATING`)
      FROM
        hotel_stats
    )
    ORDER BY
      `TOTAL_BOOKINGS_COUNT` ASC
    LIMIT 1
  )

In [0]:
%sql
-- Test out the `get_hotel_to_promote` function

USE CATALOG IDENTIFIER(:catalog);
USE DATABASE IDENTIFIER(:database);

SELECT *
  FROM get_hotel_to_promote(:hotel_class);

-- Copy the `hotel_id` value and paste it into the "hotel_to_promote" parameter at the top of this page

-- If you get a "No rows returned" message, then change the hotel_class value to the one with the next lowest performance from your previous Genie prompt

hotel_id,hotel_name,hotel_city,hotel_country,hotel_description,hotel_class,average_review_rating,review_count
H10000319,River Essen Boutique,Essen,Germany,"Affordable Retreat - Relax in our clean, comfortable rooms that provide excellent value for money. Our no-nonsense approach to hospitality ensures you get quality service at budget-friendly rates.",Economy,3.75,8


In [0]:
%sql
-- This statement creates a function that takes a HOTEL_ID as input and generates a summary of the top 3 reasons why customers enjoyed their hotel stay.

USE CATALOG IDENTIFIER(:catalog);
USE DATABASE IDENTIFIER(:database);

CREATE OR REPLACE FUNCTION
summarize_customer_reviews(input_hotel_id STRING COMMENT 'ID of the hotel to be searched')
RETURNS STRING
LANGUAGE SQL
COMMENT 'This function takes a HOTEL_ID as input and generates a summary of the top 3 reasons why customers enjoyed their hotel stay'
RETURN (
  SELECT AI_GEN(
    SUBSTRING('Extract the top 3 reasons people like the hotels based on this list of reviews:' || ARRAY_JOIN(COLLECT_LIST(REVIEW_TEXT), ' - '), 1, 80000)
  ) AS all_reviews
  FROM denormalized_hotel_bookings
    WHERE `HOTEL_ID` = input_hotel_id
    -- Try to exclude negative reviews
    AND `REVIEW_RATING` >= 3
)

In [0]:
%sql
-- Try out this function to see the top 3 summary of customer reviews for a hotel

USE CATALOG IDENTIFIER(:catalog);
USE DATABASE IDENTIFIER(:database);

SELECT summarize_customer_reviews(:hotel_to_promote)

summarize_customer_reviews(:hotel_to_promote)
"Based on the reviews, the top 3 reasons people like the hotel are: 1. **The rooftop terrace**: The beautiful views and well-designed space made it a wonderful place to unwind and relax. 2. **The dining experience**: The hotel's cuisine was world-class, showcasing local flavors and international sophistication, and exceeded all expectations. 3. **The spa facilities**: The spa was clean, reasonably priced, and provided a relaxing experience with professional services, making it a highlight of the hotel. Note that these are the only consistently positive comments in the reviews, while other aspects of the hotel received more mixed or negative feedback."


In [0]:
%sql
-- This function finds the top 10 customers who transacted the fewest bookings but showed the most interest (via page-views and page-clicks) for a given hotel class

USE CATALOG IDENTIFIER(:catalog);
USE DATABASE IDENTIFIER(:database);

CREATE OR REPLACE FUNCTION identify_target_customers (input_hotel_class STRING COMMENT 'Hotel class to filter by')
RETURNS TABLE (
  customer_email STRING,
  page_views INT,
  page_clicks INT,
  bookings INT
)
LANGUAGE SQL
COMMENT 'This function finds the top 10 customers who transacted the fewest bookings but showed the most interest (via page-views and page-clicks) for a given hotel class'
RETURN
  (
WITH filtered_clickstream AS (
  SELECT
    `CUSTOMER_EMAIL`,
    `ACTION`
  FROM
    `clickstream`
  WHERE
    `ACTION` IN ('page-view', 'page-click', 'booking-click')
),
filtered_dhb AS (
  SELECT
    `CUSTOMER_EMAIL`
  FROM
    `denormalized_hotel_bookings`
  WHERE
    `HOTEL_CLASS` = input_hotel_class
),
joined_table AS (
  SELECT
    a.`CUSTOMER_EMAIL`,
    a.`ACTION`
  FROM
    filtered_clickstream a
      JOIN filtered_dhb b
        ON a.`CUSTOMER_EMAIL` = b.`CUSTOMER_EMAIL`
),
ranked_customers AS (
  SELECT
    `CUSTOMER_EMAIL`,
    COUNT(
      CASE
        WHEN `ACTION` = 'page-view' THEN 1
      END
    ) AS `page_views`,
    COUNT(
      CASE
        WHEN `ACTION` = 'page-click' THEN 1
      END
    ) AS `page_clicks`,
    COUNT(
      CASE
        WHEN `ACTION` = 'booking-click' THEN 1
      END
    ) AS `bookings`,
    ROW_NUMBER() OVER (
        ORDER BY
          COUNT(
            CASE
              WHEN `ACTION` = 'booking-click' THEN 1
            END
          ) ASC,
          COUNT(
            CASE
              WHEN `ACTION` = 'page-view' THEN 1
            END
          ) DESC,
          COUNT(
            CASE
              WHEN `ACTION` = 'page-click' THEN 1
            END
          ) DESC
      ) AS `rank`
  FROM
    joined_table
  GROUP BY
    `CUSTOMER_EMAIL`
)
SELECT
  `CUSTOMER_EMAIL`,
  `page_views`,
  `page_clicks`,
  `bookings`
FROM
  ranked_customers
WHERE
  `rank` <= 10
ORDER BY
  `rank`
)


Executing subquery: SELECT summarize_customer_reviews(:hotel_to_promote).
Executing subquery: -- This query finds 

USE CATALOG IDENTIFIER(:catalog).
Executing subquery: USE DATABASE IDENTIFIER(:database).
Executing subquery: WITH filtered_clickstream AS (
  SELECT
    `CUSTOMER_EMAIL`,
    `ACTION`
  FROM
    `workspace`.`default`.`clickstream`
  WHERE
    `ACTION` IN ('page-view', 'page-click', 'booking-click')
),
filtered_dhb AS (
  SELECT
    `CUSTOMER_EMAIL`
  FROM
    `workspace`.`default`.`dhb`
  WHERE
    `HOTEL_CLASS` = 'Extended Stay'
),
joined_table AS (
  SELECT
    a.`CUSTOMER_EMAIL`,
    a.`ACTION`
  FROM
    filtered_clickstream a
      JOIN filtered_dhb b
        ON a.`CUSTOMER_EMAIL` = b.`CUSTOMER_EMAIL`
),
ranked_customers AS (
  SELECT
    `CUSTOMER_EMAIL`,
    COUNT(
      CASE
        WHEN `ACTION` = 'page-view' THEN 1
      END
    ) AS `page_views`,
    COUNT(
      CASE
        WHEN `ACTION` = 'page-click' THEN 1
      END
    ) AS `page_clicks`,
    COUNT(
   

In [0]:
%sql
-- Test out the identify_target_customers function

USE CATALOG IDENTIFIER(:catalog);
USE DATABASE IDENTIFIER(:database);

SELECT * FROM identify_target_customers(:hotel_class);

customer_email,page_views,page_clicks,bookings
chun.damore@gmail.com,9,3,0
abel.herzog@hotmail.com,9,0,0
emery.ondricka@gmail.com,6,1,0
paulita.leuschke@gmail.com,6,0,0
ariane.will@yahoo.com,4,5,0
daria.lemke@gmail.com,4,3,0
noel.mccullough@yahoo.com,4,3,0
elton.witting@gmail.com,4,0,0
forrest.lang@yahoo.com,4,0,0
juliann.keeling@hotmail.com,4,0,0


## Step 2: Create the AI Agent

In this step you are going to combine these three cruicial parts of our agent:

1. Tools for the Agent to use (from step 1)
2. LLM to serve as the agent's "brains"
3. System prompt that defines guidelines for the agent's tasks

In [0]:
%pip install -U databricks-sdk==0.39.0 langchain-community==0.2.16 langchain-openai==0.1.19 mlflow==2.19.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/623.0 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 623.0/623.0 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.3 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/27.4 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 27.3/27.4 MB 238.0 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/5.9 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.7 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.0 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [0]:
dbutils.library.restartPython()

In [0]:
from langchain_community.tools.databricks import UCFunctionToolkit
import pandas as pd

# Retrieve IDs from widget
warehouse_id = dbutils.widgets.get("warehouse_id")
catalog = dbutils.widgets.get("catalog")
db = dbutils.widgets.get("database")

# NOTE: If you get an error that warehouse_id, catalog, or db are not found, make sure that they are set in the widget parameters at the top of this Notebook
# If you still get an error, then you can set them manually in this cell and re-run it.

def get_tools():
    return (
        UCFunctionToolkit(warehouse_id=warehouse_id)
        # Include functions as tools using their qualified names.
        .include(f"{catalog}.{db}.*")
        .get_tools())

In [0]:
from langchain_community.chat_models.databricks import ChatDatabricks

# We're going to use llama 3.3 because it's tool-enabled and is available. Keep temp at 0 to make it more deterministic.
llm = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct",
    temperature=0.0,
    streaming=False)

In [0]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatDatabricks

#This defines our agent's system prompt. Here we can tell it what we expect it to do and guide it on using specific functions.

def get_prompt(history = [], prompt = None):
    if not prompt:
            prompt = """You are a helpful assistant for a global hotel company. Your task is to assist the marketing leadership in understanding and acting on their products and sales metrics. You can retrieve and analyze relevant data using specific functions.

            You have these three main tasks:

        1. Determine which the hotel to promote, let's call it the `selected hotel`, based on the result of the get_hotel_to_promote function based on the value of the hotel_class parameter.

        2. Use the customer review summary from the summarize_customer_reviews function and the hotel description to craft the content for a positive social marketing post to promote the `selected hotel`. Mention the hotel by its name, but do not mention that the hotel is not performing well nor mention any of its flaws.

        3. Create a list of potential customers to send the social marketing post to by using the identify_target_customers function.

        Format the results of all three tasks into a single cohesive output.

        Follow these guidelines:
        1. Call the appropriate function at each step and ensure results are retrieved before proceeding.
        2. Provide clear, coherent responses without mentioning the underlying functions.
        3. Do not reference the hotel by its ID.
        4. Mention the hotel's city and country.
        5. Answer only what the user asks for, no unnecessary information.
        6. If asked to generate Instagram posts, first determine what customers like most to ensure relevance.
        """
    return ChatPromptTemplate.from_messages([
            ("system", prompt),
            ("human", "{messages}"),
            ("placeholder", "{agent_scratchpad}"),
    ])

In [0]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

prompt = get_prompt()
tools = get_tools()
agent = create_openai_tools_agent(llm, tools, prompt)

def model_parsing_error_handler(e):
    print(f"This error occurred during model parsing: {e}")

#Put the pieces together to create our Agent
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=model_parsing_error_handler)

In [0]:
from operator import itemgetter
from langchain.schema.runnable import RunnableLambda
from langchain_core.output_parsers import StrOutputParser

# Very basic chain that allows us to pass the input (messages) into the Agent and collect the (output) as a string

agent_str = ({ "messages": itemgetter("messages")} | agent_executor | itemgetter("output") | StrOutputParser())


In [0]:
import logging
# Uncomment the line below if you are experiencing errors
# logging.basicConfig(level=logging.DEBUG)

answer = ""

# Retrieve the hotel class from the widget
hotel_class = dbutils.widgets.get("hotel_class")

# Construct the message
message = f"Use the available tools to find the ideal hotel to promote from the {hotel_class} class, then create a positive social marketing post to promote it and lastly find a list of prospective customers to send the post to. Mention the hotel by its name and location."

# Invoke the agent with the constructed message
try:
    answer = agent_str.invoke({"messages": message})
except AssertionError as e:
    logging.error("AssertionError: Statement execution failed but no error message was provided.")
    raise e

display(answer)

INFO:py4j.clientserver:Received command c on object id p0
DEBUG:py4j.clientserver:Command to send: c
o409
logExecuteCommandEvent
sDATABRICKS_SHELL_DO_EXECUTE_START
n
e

DEBUG:py4j.clientserver:Answer received: !yv
DEBUG:py4j.clientserver:Command to send: c
o409
logExecuteCommandEvent
sSWAP_REMOTE_SPARK_CLIENT
n
e

DEBUG:py4j.clientserver:Answer received: !yv
DEBUG:py4j.clientserver:Command to send: c
o420
longQueryExecutionWarningEnabled
e

DEBUG:py4j.clientserver:Answer received: !ybtrue
DEBUG:py4j.clientserver:Command to send: c
t
getCurrentBindings
e

DEBUG:py4j.clientserver:Answer received: !yao503
DEBUG:py4j.clientserver:Command to send: c
o407
getArgument
shotel_class
ro503
e

DEBUG:py4j.clientserver:Answer received: !ysEconomy
DEBUG:databricks.sdk:/home/spark-12ac9534-27ef-42ca-9874-e5/.databrickscfg does not exist
DEBUG:databricks.sdk:Attempting to configure auth: pat
DEBUG:databricks.sdk:Attempting to configure auth: basic
DEBUG:databricks.sdk:Attempting to configure auth: met


Invoking: `workspace__default__hotel_to_promote` with `{'input_hotel_class': 'Economy'}`




DEBUG:urllib3.connectionpool:https://dbc-ccf41f92-240c.cloud.databricks.com:443 "POST /api/2.0/sql/statements/ HTTP/1.1" 200 None
DEBUG:databricks.sdk:POST /api/2.0/sql/statements/
> {
>   "byte_limit": 4096,
>   "parameters": [
>     {
>       "name": "input_hotel_class",
>       "type": "string",
>       "value": "**REDACTED**"
>     }
>   ],
>   "row_limit": 100,
>   "statement": "SELECT * FROM workspace.default.hotel_to_promote(:input_hotel_class)",
>   "wait_timeout": "30s",
>   "warehouse_id": "77ad79eadc0d646b"
> }
< 200 OK
< {
<   "manifest": {
<     "chunks": [
<       {
<         "chunk_index": 0,
<         "row_count": 1,
<         "row_offset": 0
<       }
<     ],
<     "format": "JSON_ARRAY",
<     "schema": {
<       "column_count": 8,
<       "columns": [
<         {
<           "name": "hotel_id",
<           "position": 0,
<           "type_name": "STRING",
<           "type_text": "STRING"
<         },
<         "... (7 additional elements)"
<       ]
<     },
<     

{"format": "CSV", "value": "hotel_id,hotel_name,hotel_city,hotel_country,hotel_description,hotel_class,average_review_rating,review_count\nH10000319,River Essen Boutique,Essen,Germany,\"Affordable Retreat - Relax in our clean, comfortable rooms that provide excellent value for money. Our no-nonsense approach to hospitality ensures you get quality service at budget-friendly rates.\",Economy,3.75,8\n", "truncated": false}

DEBUG:py4j.clientserver:Command to send: m
d
o505
e

DEBUG:py4j.clientserver:Answer received: !yv
DEBUG:urllib3.connectionpool:https://dbc-ccf41f92-240c.cloud.databricks.com:443 "POST /serving-endpoints/databricks-meta-llama-3-3-70b-instruct/invocations HTTP/1.1" 200 None
DEBUG:databricks.sdk:POST /serving-endpoints/databricks-meta-llama-3-3-70b-instruct/invocations
> [raw stream]
< 200 OK
< [raw stream]


...

{"format": "CSV", "value": "customer_email,page_views,page_clicks,bookings\nchun.damore@gmail.com,9,3,0\nabel.herzog@hotmail.com,9,0,0\nemery.ondricka@gmail.com,6,1,0\npaulita.leuschke@gmail.com,6,0,0\nariane.will@yahoo.com,4,5,0\ndaria.lemke@gmail.com,4,3,0\nnoel.mccullough@yahoo.com,4,3,0\nforrest.lang@yahoo.com,4,0,0\nelton.witting@gmail.com,4,0,0\njuliann.keeling@hotmail.com,4,0,0\n", "truncated": false}

DEBUG:py4j.clientserver:Command to send: m
d
o513
e

DEBUG:py4j.clientserver:Answer received: !yv
DEBUG:urllib3.connectionpool:https://dbc-ccf41f92-240c.cloud.databricks.com:443 "POST /serving-endpoints/databricks-meta-llama-3-3-70b-instruct/invocations HTTP/1.1" 200 None
DEBUG:databricks.sdk:POST /serving-endpoints/databricks-meta-llama-3-3-70b-instruct/invocations
> [raw stream]
< 200 OK
< [raw stream]
DEBUG:py4j.clientserver:Command to send: c
o419
logServerlessKeepaliveEvent
n
n
e

DEBUG:py4j.clientserver:Answer received: !yv
DEBUG:py4j.clientserver:Command to send: c
o419
logServerlessKeepaliveEvent
n
n
e

DEBUG:py4j.clientserver:Answer received: !yv


Based on the results, the ideal hotel to promote from the Economy class is the River Essen Boutique in Essen, Germany. 

Here's a positive social marketing post to promote it: "Looking for an affordable retreat in the heart of Germany? Look no further than the River Essen Boutique in Essen! Our hotel offers clean and comfortable rooms at budget-friendly rates, a beautiful rooftop terrace with stunning views, a world-class dining experience, and a relaxing spa facility. Come and experience it for yourself! 

The post will be sent to the following list of prospective customers: 
chun.damore@gmail.com
abel.herzog@hotmail.com
emery.ondricka@gmail.com
paulita.leuschke@gmail.com
ariane.will@yahoo.com
daria.lemke@gmail.com
noel.mccullough@yahoo.com
forrest.lang@yahoo.com
elton.witting@gmail.com
juliann.keeling@hotmail.com

> Finished chain.


'Based on the results, the ideal hotel to promote from the Economy class is the River Essen Boutique in Essen, Germany. \n\nHere\'s a positive social marketing post to promote it: "Looking for an affordable retreat in the heart of Germany? Look no further than the River Essen Boutique in Essen! Our hotel offers clean and comfortable rooms at budget-friendly rates, a beautiful rooftop terrace with stunning views, a world-class dining experience, and a relaxing spa facility. Come and experience it for yourself! \n\nThe post will be sent to the following list of prospective customers: \nchun.damore@gmail.com\nabel.herzog@hotmail.com\nemery.ondricka@gmail.com\npaulita.leuschke@gmail.com\nariane.will@yahoo.com\ndaria.lemke@gmail.com\nnoel.mccullough@yahoo.com\nforrest.lang@yahoo.com\nelton.witting@gmail.com\njuliann.keeling@hotmail.com'

DEBUG:py4j.clientserver:Command to send: c
o0
contains
sspark.databricks.workspaceUrl
e

DEBUG:py4j.clientserver:Answer received: !ybfalse
DEBUG:py4j.clientserver:Command to send: c
o0
contains
sspark.databricks.clusterUsageTags.sparkVersion
e

DEBUG:py4j.clientserver:Answer received: !ybtrue
DEBUG:py4j.clientserver:Command to send: c
o0
get
sspark.databricks.clusterUsageTags.sparkVersion
e

DEBUG:py4j.clientserver:Answer received: !ysclient.2.5-scala2.12
DEBUG:py4j.clientserver:Command to send: c
o409
logExecuteCommandEvent
sDATABRICKS_SHELL_DO_EXECUTE_END
n
e

DEBUG:py4j.clientserver:Answer received: !yv
